In [5]:

from itertools import product

import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf624892aef431149a47ecb9e4ba5baf35566b')

In [6]:
m = folium.Map(location=[28.2268,77.1635], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in delhi
coordinates = [
    [77.1245, 28.8432], [77.2210, 28.7568], [77.2128, 28.6784], [77.0379, 28.6139], [77.2698, 28.5012],
    [77.2387, 28.6543], [77.0934, 28.4873], [77.1245, 28.7345], [77.1612, 28.5829], [77.3172, 28.6791],
    [77.1034, 28.7154], [77.2749, 28.5276], [77.0964, 28.6487], [77.2837, 28.4921], [77.0882, 28.4890],
    [77.0205, 28.7102], [77.2268, 28.6635], [77.0987, 28.4339], [77.3182, 28.4211], [77.1025, 28.7041],
    [77.2148, 28.5394], [77.1689, 28.4872], [77.3230, 28.6713], [77.2139, 28.7258], [77.2436, 28.5774],
    [77.2780, 28.6294], [77.0264, 28.4768], [77.2043, 28.5345], [77.0645, 28.7261], [77.2507, 28.5412],
    [77.0938, 28.6176], [77.3159, 28.5883], [77.2069, 28.5192], [77.1214, 28.6401], [77.2819, 28.5598],
    [77.0269, 28.4531], [77.3041, 28.4928], [77.2583, 28.7089], [77.1023, 28.4439], [77.0689, 28.6841],
    [77.0361, 28.4823], [77.1124, 28.4234], [77.0225, 28.4672], [77.0498, 28.7589], [77.2654, 28.7152],
    [77.1524, 28.6883], [77.2198, 28.6429], [77.0923, 28.5298], [77.2098, 28.5031], [77.0986, 28.7246],
    [77.1345, 28.7892], [77.2451, 28.5678], [77.2867, 28.5943], [77.1239, 28.4682], [77.1578, 28.6361],
    [77.0982, 28.5195], [77.1768, 28.7339], [77.2891, 28.4908], [77.0564, 28.5132], [77.2145, 28.4569],
    [77.0314, 28.6775], [77.1923, 28.5406], [77.2738, 28.6258], [77.1879, 28.5821], [77.1642, 28.4294],
    [77.1126, 28.5918], [77.2378, 28.6792], [77.0829, 28.7040], [77.1476, 28.5263], [77.2641, 28.5987],
    [77.0361, 28.4823], [77.1124, 28.4234], [77.0225, 28.4672], [77.0498, 28.7589], [77.2654, 28.7152],
    [77.3124, 28.7041], [77.3035, 28.5672], [77.2891, 28.4599], [77.2156, 28.6784], [77.1987, 28.5912],
    [77.2765, 28.6451], [77.1623, 28.7243], [77.1345, 28.5643], [77.2456, 28.4325], [77.3241, 28.6892],
    [77.1762, 28.7095], [77.1912, 28.5017], [77.1584, 28.6334], [77.3220, 28.5432], [77.2105, 28.4732],
    [77.1876, 28.6819], [77.2634, 28.5287], [77.2413, 28.6134], [77.2299, 28.4567], [77.1725, 28.4845],
    [77.2945, 28.5987], [77.3077, 28.6691], [77.1890, 28.4876], [77.2184, 28.5403], [77.1699, 28.5721]
    
]


print(len(coordinates))
print(type(coordinates))
full_durations = [[] for _ in range(100)]  # 100 empty lists for durations
full_distances = [[] for _ in range(100)]  # 100 empty lists for distances

# Process in batches of 25
batch_size = 25
for i in range(0, len(coordinates), batch_size):
    batch = coordinates[i:i + batch_size]
    # The API call is made with the batch as the source and all coordinates as destinations
    matrix = client.distance_matrix(
        locations=batch + coordinates,  # Batch first, then all coordinates
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        sources=list(range(len(batch))),  # Sources: indices of batch
        destinations=list(range(len(batch), len(batch) + len(coordinates)))  # Destinations: indices of all locations
    )
    
    # Distribute the response into the full matrices
    for j, row in enumerate(matrix['durations']):
        full_durations[i + j] = row[len(batch):]  # Append the durations skipping the first batch-size elements
    
    if 'distances' in matrix:
        for j, row in enumerate(matrix['distances']):
            full_distances[i + j] = row[len(batch):]  # Append the distances similarly

# Adding markers for all locations on the map
for marker in coordinates:
    folium.Marker(location=[marker[1], marker[0]]).add_to(m)

# Display the map in Jupyter Notebook (if you're using it)
# m

# Output the full duration and distance matrices
print("Full Time Matrix (minutes):")
for row in full_durations:
    print([f"{value / 60:.2f}" for value in row])

print("Full Distance Matrix (meters):")
for row in full_distances:
    print(row)

m

100
<class 'list'>
Full Time Matrix (minutes):
['36.35', '57.13', '51.50', '22.25', '46.91', '30.94', '43.99', '51.27', '32.73', '46.14', '60.04', '51.99', '27.28', '60.26', '26.50', '59.22', '63.59', '58.01', '18.87', '29.20', '25.70', '33.22', '47.31', '57.70', '24.62', '12.23', '42.54', '44.06', '60.77', '36.67', '46.89', '20.79', '55.20', '53.45', '66.15', '29.23', '50.82', '39.37', '45.49', '66.78', '40.19', '33.89', '26.72', '48.04', '40.61', '59.22', '63.59', '58.01', '18.87', '29.20', '33.53', '45.72', '56.38', '27.33', '43.40', '35.26', '18.45', '41.90', '62.74', '34.65', '22.35', '56.19', '37.26', '47.55', '67.64', '27.29', '50.98', '35.06', '74.47', '60.59', '40.24', '37.39', '55.99', '47.42', '43.58']
['26.30', '54.06', '41.45', '27.44', '36.86', '29.05', '33.94', '41.22', '30.85', '36.09', '55.20', '41.94', '17.23', '54.26', '24.62', '55.09', '57.58', '55.54', '30.29', '19.15', '23.69', '24.08', '44.45', '47.65', '24.58', '17.54', '32.50', '34.01', '54.77', '32.04', '40.88

In [18]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


# In[5]:


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = full_durations
    data['time_windows'] = [
(0, 5),
(7, 12),
(10, 15),
(16, 18),
(10, 13),
(0, 5),
(5, 10),
(0, 4),
(5, 10),
(0, 3),
(10, 16),
(10, 15),
(0, 5),
(5, 10),
(7, 8),
(10, 15),
(11, 15),
(13, 18),
(12, 19),
(8, 9),
(1, 8),
(12, 22),
(1, 9),
(6, 9),
(7, 8),
(7, 16),
(4, 6),
(13, 16),
(13, 19),
(4, 8),
(9, 13),
(7, 16),
(3, 9),
(4, 13),
(4, 11),
(7, 16),
(7, 10),
(10, 11),
(9, 14),
(14, 20),
(10, 17),
(15, 18),
(10, 16),
(4, 5),
(8, 15),
(8, 14),
(6, 16),
(11, 19),
(15, 20),
(11, 13),
(11, 14),
(7, 9),
(6, 16),
(8, 17),
(3, 7),
(15, 19),
(2, 3),
(7, 14),
(2, 4),
(2, 4),
(4, 12),
(7, 9),
(11, 12),
(9, 15),
(5, 13),
(15, 16),
(7, 10),
(6, 12),
(12, 21),
(11, 15),
(12, 21),
(2, 12),
(7, 13),
(2, 4),
(7, 15),
(2, 7),
(10, 18),
(5, 15),
(0, 1),
(9, 11),
(9, 18),
(1, 7),
(13, 21),
(11, 20),
(10, 18),
(6, 12),
(8, 12),
(4, 10),
(14, 22),
(1, 5),
(10, 15),
(7, 14),
(5, 7),
(7, 11),
(0, 1),
(3, 11),
(7, 17),
(8, 11),
(11, 12),
(0, 3),
    ]

    data['distance_matrix'] = full_distances

    print(len(data['time_matrix']))
    data["demands"] = [0, 8, 2, 3, 3, 7, 4, 8, 1, 2, 6, 4, 3, 1, 6, 3, 5, 1, 5, 3, 4, 4, 1, 2, 6, 7, 8, 8, 7, 8, 7, 3, 8, 3, 5, 7, 3, 3, 8, 7, 5, 5, 8, 2, 6, 4, 6, 8, 2, 7, 7, 6, 2, 6, 2, 5, 2, 8, 3, 5, 4, 5, 3, 7, 5, 6, 7, 5, 1, 2, 3, 7, 2, 8, 2, 7, 5, 8, 3, 3, 6, 1, 2, 8, 8, 7, 6, 8, 7, 6, 7, 8, 7, 7, 6, 3, 3, 5, 8, 8]
    data["vehicle_capacities"] = [70, 50,50 , 50, 50,50,50,50,60,50]
    data["num_vehicles"] = 10
    data['depot'] = 0
    assert data['num_vehicles'] == len(data['vehicle_capacities'])
    assert len(data['time_matrix']) == len(data['distance_matrix'])
    assert len(data['time_matrix']) == len(data['time_windows'])
    assert len(data['time_matrix']) == len(data['demands'])
    return data



In [19]:

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes)
    # Print routes
    time_dimension = routing.GetDimensionOrDie('Time')
    distance_dimension = routing.GetDimensionOrDie('Distance')
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    total_time = 0
    total_distance = 0
    total_load=0
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            time_var = time_dimension.CumulVar(index)
            distance_var = distance_dimension.CumulVar(index)
            capacity_var = capacity_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) Distance:{3} Load:{4}  -> '.format(
                node_index,
                solution.Min(time_var), solution.Max(time_var),
                solution.Value(distance_var),
                solution.Value(capacity_var))
            index = solution.Value(routing.NextVar(index))
        node_index = manager.IndexToNode(index)
        time_var = time_dimension.CumulVar(index)
        distance_var = distance_dimension.CumulVar(index)
        capacity_var = capacity_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2}) Distance:{3} Load:{4})\n'.format(
            manager.IndexToNode(index),
            solution.Min(time_var), solution.Max(time_var),
            solution.Value(distance_var),
            solution.Value(capacity_var))
        plan_output += 'Time of the route: {}min\n'.format(solution.Min(time_var))
        plan_output += 'Distance of the route: {}m\n'.format(solution.Value(distance_var))
        plan_output += 'Load of the route: {}\n'.format(solution.Value(capacity_var))
        print(plan_output)
        total_time += solution.Min(time_var)
        total_distance += solution.Value(distance_var)
        total_load += solution.Value(capacity_var)
    print('Total time of all routes: {}min'.format(total_time))
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))



In [23]:

def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
            len(data['distance_matrix']),
            data['num_vehicles'],
            data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

   # Allow to drop nodes.
    penalty = 1_000_000
    for node in range(1, len(data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty )

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    distance_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        distance_callback_index,
        0,  # no slack
        100,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Add Time Windows constraint.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    time_callback_index = routing.RegisterTransitCallback(time_callback)

    time = 'Time'
    routing.AddDimension(
        time_callback_index,
        100,  # allow waiting time
        100,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    #search_parameters.log_search = True
    search_parameters.time_limit.FromSeconds(60)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)
    else:
        print('no solution found')

if __name__ == '__main__':
    main()


100
Objective: 0
Dropped nodes:
Route for vehicle 0:
0 Time(0,0) Distance:0 Load:0  -> 12 Time(0,3) Distance:0 Load:0  -> 9 Time(0,3) Distance:0 Load:3  -> 7 Time(0,4) Distance:0 Load:5  -> 6 Time(5,5) Distance:0 Load:13  -> 5 Time(5,5) Distance:0 Load:17  -> 1 Time(7,12) Distance:0 Load:24  -> 4 Time(10,13) Distance:0 Load:32  -> 3 Time(16,16) Distance:0 Load:35  -> 0 Time(16,16) Distance:0 Load:38)
Time of the route: 16min
Distance of the route: 0m
Load of the route: 38

Route for vehicle 1:
0 Time(0,0) Distance:0 Load:0  -> 29 Time(4,6) Distance:0 Load:0  -> 26 Time(4,6) Distance:0 Load:8  -> 24 Time(7,8) Distance:0 Load:16  -> 23 Time(7,8) Distance:0 Load:22  -> 22 Time(7,8) Distance:0 Load:24  -> 20 Time(7,8) Distance:0 Load:25  -> 19 Time(8,8) Distance:0 Load:29  -> 14 Time(8,8) Distance:0 Load:32  -> 13 Time(8,10) Distance:0 Load:38  -> 11 Time(10,10) Distance:0 Load:39  -> 10 Time(10,10) Distance:0 Load:43  -> 8 Time(10,10) Distance:0 Load:49  -> 0 Time(10,10) Distance:0 Load:5